# Module 11 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html=browser.html
soup=BeautifulSoup(html,'html.parser')

In [ ]:
# Extract all rows of data
rows=soup.find_all('tr',class_='data-row')


### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
row_list=[]

# Loop through the scraped data to create a list of rows
# I used to https://stackoverflow.com/questions/61457049/how-can-i-remove-the-td-tag-from-the-data-frame 
                                
for row in rows:
    row_data = [td.get_text() for td in row.find_all('td')]
    row_list.append(row_data)

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_data=pd.DataFrame(row_list)
mars_data.columns=['id','terristrial_date','sol','ls','month','min_temp','pressure']

In [ ]:
# Confirm DataFrame was created successfully
mars_data.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_data.info()

In [ ]:
# Change data types for data analysis
mars_data = mars_data.astype({'id':int,'sol':int, 'ls':int, 'month':int, 'min_temp':float, 'pressure':float})

In [ ]:
#Change datetime datatype
mars_data['terristrial_date'] = pd.to_datetime(mars_data['terristrial_date'])

In [ ]:
# Confirm type changes were successful by examining data types again
mars_data.info()

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars? (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html)
mars_data['month'].value_counts().sort_index()

In [ ]:
# 2. How many Martian days' worth of data are there?
mars_data['sol'].nunique()

In [ ]:
# 3. What is the average low temperature by month?
temp_month=mars_data[['month', 'min_temp']]
temp_month = temp_month.groupby(['month']).mean()
temp_month

In [ ]:
# Plot the average temperature by month
temp_month.plot(kind="bar")
plt.title("Mars Average Monthly Temperature")
plt.xlabel("Month")
plt.ylabel("Temperature in Celsius")
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
sorted_temps=temp_month.sort_values('min_temp')
sorted_temps.plot(kind="bar")
plt.title("Mars Average Monthly Temperature (Sorted)")
plt.xlabel("Month")
plt.ylabel("Temperature in Celsius")
plt.show()

In [ ]:
# 4. Average pressure by Martian month
pressure_month=mars_data[['month', 'pressure']]
pressure_month = pressure_month.groupby(['month']).mean()
sorted_pressure= pressure_month.sort_values('pressure')
sorted_pressure

In [ ]:
# Plot the average pressure by month
sorted_pressure.plot(kind="bar")
plt.title("Mars Average Monthly Pressure")
plt.xlabel("Month")
plt.ylabel("Atmospheric Pressure")
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
#I used https://stackoverflow.com/questions/61321813/python-set-days-from-a-date-as-index-for-a-plot
mars_data['count_earth_days'] = (mars_data['terristrial_date']-mars_data['terristrial_date'].min())/np.timedelta64(1,'D')
mars_data.plot(x='count_earth_days',y='min_temp')
plt.xlabel("Number of Terristrial Days")
plt.ylabel("Minimum temperature in Celsius")
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_data.to_csv("output/mars_data.csv",index=True)

In [ ]:
browser.quit()